In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTENC
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cad_df = pd.read_csv('final_data.csv')
cad_df['Cath'].value_counts()

Cath
0    216
1     87
Name: count, dtype: int64

# ***Recursive Feature Elimination (RFE)***

In [ ]:
#Separate features and target columns
X_feature = cad_df.drop('Cath', axis=1)
y_target = cad_df['Cath']

RF_mdl = RandomForestClassifier(random_state=1)
select = RFE(RF_mdl, n_features_to_select=30)  # RFE with 30 features selected
select.fit(X_feature, y_target)
selected_features = X_feature.columns[select.support_]
X_rfe_chosen = X_feature[selected_features]

print("Selected features with RFE:", selected_features.tolist())
print("\nDataframe with only the selected features:")
print(X_rfe_chosen.head(5))

Selected features with RFE: ['Age', 'Weight', 'Length', 'BMI', 'DM', 'HTN', 'BP', 'PR', 'Typical Chest Pain', 'Dyspnea', 'Atypical', 'Nonanginal', 'Tinversion', 'FBS', 'CR', 'TG', 'LDL', 'HDL', 'BUN', 'ESR', 'HB', 'K', 'Na', 'WBC', 'Lymph', 'Neut', 'PLT', 'EF-TTE', 'Region RWMA', 'VHD']

Dataframe with only the selected features:
   Age  Weight  Length        BMI  DM  HTN   BP   PR  Typical Chest Pain  \
0   53      90     175  29.387755   0    1  110   80                   0   
1   67      70     157  28.398718   0    1  140   80                   1   
2   54      54     164  20.077335   0    0  100  100                   1   
3   66      67     158  26.838648   0    1  100   80                   0   
4   50      87     153  37.165193   0    1  110   80                   0   

   Dyspnea  ...    HB    K   Na    WBC  Lymph  Neut  PLT  EF-TTE  Region RWMA  \
0        0  ...  15.6  4.7  141   5700     39    52  261      50            0   
1        0  ...  13.9  4.7  156   7700     38    

# ***Data splitting***

In [ ]:
# 80% data for training and 20% data for testing
X_train, X_test, y_train, y_test = train_test_split(X_rfe_chosen, y_target, test_size=0.2, random_state=1)

print(f"The training set contains {X_train.shape[0]} patient records and {X_train.shape[1]} features.")
print(f"The test set contains {X_test.shape[0]} patient records, each with {X_test.shape[1]} features.")

The training set contains 242 patient records and 30 features.
The test set contains 61 patient records, each with 30 features.


# ***SMOTENC***

In [ ]:
categorical_cols = [col for col in X_train.columns if X_train[col].nunique() <= 4]
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=1)
X_balanced, y_balanced = smote_nc.fit_resample(X_train, y_train)

print("Imbalanced distribution:", Counter(y_train))
print("Balanced distribution:", Counter(y_balanced))

Imbalanced distribution: Counter({0: 170, 1: 72})
Balanced distribution: Counter({0: 170, 1: 170})


Applied SMOTENC balancing only to the testing set

In [ ]:
X_balanced

Age  Weight  Length        BMI  DM  HTN   BP  PR  Typical Chest Pain  \
0     41     120     183  35.832661   1    1  130  70                   0   
1     79      63     160  24.609375   0    0  130  60                   1   
2     81      74     165  27.180900   0    1  140  70                   1   
3     56      62     154  26.142688   0    0  110  70                   0   
4     60      64     168  22.675737   0    0  130  70                   0   
..   ...     ...     ...        ...  ..  ...  ...  ..                 ...   
335   48      79     172  26.819134   0    1  144  82                   0   
336   49      75     172  25.370383   0    1  136  70                   0   
337   50      69     148  31.295583   0    0  129  79                   0   
338   45      71     172  23.932806   0    0  113  71                   0   
339   58      66     163  24.812977   0    1  137  70                   0   

     Dyspnea  ...         HB         K   Na   WBC  Lymph  Neut  PLT  EF-TTE  \
0          0  ...  15.200000  5.000000  141  6900     41    50  252      55   
1          0  ...  14.800000  4.800000  145  7200     36    52  240      40   
2          1  ...  11.900000  4.500000  142  6000     37    50  280      45   
3          0  ...  13.900000  4.500000  138  5700     37    55  240      45   
4          1  ...  10.800000  4.000000  139  4100     42    45  156      55   
..       ...  ...        ...       ...  ...   ...    ...   ...  ...     ...   
335        0  ...  14.779674  3.900000  147  5651     36    57  190      42   
336        1  ...  15.383767  3.800000  143  6930     29    59  223      49   
337        0  ...  13.356348  4.431747  142  7507     34    54  189      35   
338        0  ...  13.023017  4.525551  143  8462     49    44  267      54   
339        1  ...  13.278479  4.257544  142  6800     34    55  235      51   

     Region RWMA  VHD  
0              0    1  
1              1    3  
2              0    1  
3              0    0  
4              0    1  
..           ...  ...  
335            0    3  
336            0    1  
337            0    1  
338            0    1  
339            0    1  

[340 rows x 30 columns]

# ***Random Forest Classifier (RF)***

In [ ]:
param_random = {'n_estimators': [50, 100, 150],
                'max_depth': [2, 3, 4],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 4, 8],
                'max_features': ['sqrt', 'log2']}

RF_mdl = RandomForestClassifier()
RF_randomcv = RandomizedSearchCV(RF_mdl, param_random, n_iter=20, cv=5)
RF_randomcv.fit(X_balanced, y_balanced)
RF_param = RF_randomcv.best_params_
print("Best cross-validation score: {:.2f}".format(RF_randomcv.best_score_))
print("Optimal parameters of RF classifier: ", RF_param)

#Training RF model with optimal parameters
RF_best_mdl = RandomForestClassifier(**RF_param)
RF_best_mdl.fit(X_balanced, y_balanced)

# Training performance
train_pred = RF_best_mdl.predict(X_balanced)
training_acc = RF_best_mdl.score(X_balanced, y_balanced)
print(f"Training phase accuracy of RF classifier: {training_acc:.4f}")
print("=== Training Report ===")
print(classification_report(y_balanced, train_pred))

# Testing performance
test_pred = RF_best_mdl.predict(X_test)
print("=== Testing Report ===")
print(classification_report(y_test, test_pred))

Best cross-validation score: 0.92
Optimal parameters of RF classifier:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 4}
Training phase accuracy of RF classifier: 0.9441
=== Training Report ===
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       170
           1       0.95      0.94      0.94       170

    accuracy                           0.94       340
   macro avg       0.94      0.94      0.94       340
weighted avg       0.94      0.94      0.94       340

=== Testing Report ===
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        46
           1       0.58      0.73      0.65        15

    accuracy                           0.80        61
   macro avg       0.74      0.78      0.76        61
weighted avg       0.82      0.80      0.81        61



In [ ]:
matrix_train = confusion_matrix(y_balanced, train_pred)
matrix_test = confusion_matrix(y_test, test_pred)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Plotting the training confusion matrix
disp_val = ConfusionMatrixDisplay(confusion_matrix=matrix_train, display_labels=[0, 1])
disp_val.plot(ax=axs[0], cmap='Purples', colorbar=False)
axs[0].set_title('RF Classifier - Training')

# Plotting the testing confusion matrix
disp_test = ConfusionMatrixDisplay(confusion_matrix=matrix_test, display_labels=[0, 1])
disp_test.plot(ax=axs[1], cmap='Purples', colorbar=False)
axs[1].set_title('RF Classifier - Testing')

plt.tight_layout()
plt.show()

# ***XGBoost (XGB)***

In [ ]:
param_random = {'n_estimators': [50, 80, 100],
                'max_depth': [2, 3, 4],
                'learning_rate': [0.01, 0.02, 0.05],
                'subsample': [0.6, 0.7, 0.8]}

XGB_mdl = XGBClassifier()
XGB_randomcv = RandomizedSearchCV(XGB_mdl, param_random, n_iter=20, cv=5)
XGB_randomcv.fit(X_balanced, y_balanced)
XGB_param = XGB_randomcv.best_params_
print("Best cross-validation score: {:.2f}".format(XGB_randomcv.best_score_))
print("Optimal parameters of XGB classifier: ", XGB_param)

#Training XGB model with optimal parameters
XGB_best_mdl = XGBClassifier(**XGB_param)
XGB_best_mdl.fit(X_balanced, y_balanced)

# Training performance
train_pred = XGB_best_mdl.predict(X_balanced)
training_acc = XGB_best_mdl.score(X_balanced, y_balanced)
print(f"Training phase accuracy of XGB classifier: {training_acc:.4f}")
print("=== Training Report ===")
print(classification_report(y_balanced, train_pred))

# Testing performance
test_pred = XGB_best_mdl.predict(X_test)
print("=== Testing Report ===")
print(classification_report(y_test, test_pred))

Best cross-validation score: 0.92
Optimal parameters of XGB classifier:  {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01}
Training phase accuracy of XGB classifier: 0.9294
=== Training Report ===
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       170
           1       0.91      0.95      0.93       170

    accuracy                           0.93       340
   macro avg       0.93      0.93      0.93       340
weighted avg       0.93      0.93      0.93       340

=== Testing Report ===
              precision    recall  f1-score   support

           0       0.92      0.74      0.82        46
           1       0.50      0.80      0.62        15

    accuracy                           0.75        61
   macro avg       0.71      0.77      0.72        61
weighted avg       0.82      0.75      0.77        61



In [ ]:
matrix_train = confusion_matrix(y_balanced, train_pred)
matrix_test = confusion_matrix(y_test, test_pred)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

disp_val = ConfusionMatrixDisplay(confusion_matrix=matrix_train, display_labels=[0, 1])
disp_val.plot(ax=axs[0], cmap='Purples', colorbar=False)
axs[0].set_title('XGB Classifier - Training')

disp_test = ConfusionMatrixDisplay(confusion_matrix=matrix_test, display_labels=[0, 1])
disp_test.plot(ax=axs[1], cmap='Purples', colorbar=False)
axs[1].set_title('XGB Classifier - Testing')

plt.tight_layout()
plt.show()

# ***Gradient Boosting (GB)***

In [ ]:
param_random = {'n_estimators': [50, 100, 150],
                'max_depth': [2, 3, 4],
                'learning_rate': [0.005, 0.001, 0.015],
                'subsample': [0.6, 0.7, 0.8],
                'min_samples_split': [10, 12],
                'min_samples_leaf': [6, 8, 10]}

GB_mdl = GradientBoostingClassifier()
GB_randomcv = RandomizedSearchCV(GB_mdl, param_random, n_iter=20, cv=5)
GB_randomcv.fit(X_balanced, y_balanced)
GB_param = GB_randomcv.best_params_
print("Best cross-validation score: {:.2f}".format(GB_randomcv.best_score_))
print("Optimal parameters of GB classifier: ", GB_param)

#Training GB model with optimal parameters
GB_best_mdl = GradientBoostingClassifier(**GB_param)
GB_best_mdl.fit(X_balanced, y_balanced)

# Training performance
train_pred = GB_best_mdl.predict(X_balanced)
training_acc = GB_best_mdl.score(X_balanced, y_balanced)
print(f"Training phase accuracy of GB classifier: {training_acc:.4f}")
print("=== Training Report ===")
print(classification_report(y_balanced, train_pred))

# Testing performance
test_pred = GB_best_mdl.predict(X_test)
print("=== Testing Report ===")
print(classification_report(y_test, test_pred))

Best cross-validation score: 0.92
Optimal parameters of GB classifier:  {'subsample': 0.6, 'n_estimators': 150, 'min_samples_split': 12, 'min_samples_leaf': 8, 'max_depth': 4, 'learning_rate': 0.015}
Training phase accuracy of GB classifier: 0.9647
=== Training Report ===
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       170
           1       0.97      0.96      0.96       170

    accuracy                           0.96       340
   macro avg       0.96      0.96      0.96       340
weighted avg       0.96      0.96      0.96       340

=== Testing Report ===
              precision    recall  f1-score   support

           0       0.92      0.76      0.83        46
           1       0.52      0.80      0.63        15

    accuracy                           0.77        61
   macro avg       0.72      0.78      0.73        61
weighted avg       0.82      0.77      0.78        61



In [ ]:
matrix_train = confusion_matrix(y_balanced, train_pred)
matrix_test = confusion_matrix(y_test, test_pred)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

disp_val = ConfusionMatrixDisplay(confusion_matrix=matrix_train, display_labels=[0, 1])
disp_val.plot(ax=axs[0], cmap='Purples', colorbar=False)
axs[0].set_title('GB Classifier - Training')

disp_test = ConfusionMatrixDisplay(confusion_matrix=matrix_test, display_labels=[0, 1])
disp_test.plot(ax=axs[1], cmap='Purples', colorbar=False)
axs[1].set_title('GB Classifier - Testing')

plt.tight_layout()
plt.show()